<a href="https://colab.research.google.com/github/brunogresende/ppca-aed/blob/main/Aloca%C3%A7%C3%A3o_de_Pessoas_em_Trabalho_Remoto_Scheduling_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#!pip install -U pulp
import pulp

n = 20
tAJr = 30
tASr = 9
tE = 2
t = tAJr + tASr + tE

var_L = pulp.LpVariable.dicts("Leave", (range(n), range(t)), 0, 1, "Binary")
var_E = pulp.LpVariable.dicts("HomeEsp", (range(n), range(tE)), 0, 1, "Binary")
var_AJr = pulp.LpVariable.dicts("HomeAJr", (range(n), range(tAJr)), 0, 1, "Binary")
var_ASr = pulp.LpVariable.dicts("HomeASr", (range(n), range(tASr)), 0, 1, "Binary")

#objective function
obj = None
for i in range(n):
  for j in range(tE):
    obj += var_E[i][j]
    for jj in range(tAJr):
      obj += var_AJr[i][jj]
      for jjj in range(tASr):
        obj += var_ASr[i][jjj]

problem = pulp.LpProblem("schedule", pulp.LpMaximize)
problem += obj

#constraint 1
c = None
for i in range(n):
  for j in range(tE):
    c += var_E[i][j]
    for jj in range(tAJr):
      c += var_AJr[i][jj]
      for jjj in range(tASr):
        c += var_ASr[i][jjj]
        problem += c == 1

status = problem.solve()
print(status)
print("Status", pulp.LpStatus[status])

1
Status Optimal
